In [1]:
# !pip install transformers==4.46.2 datasets==3.1.0 numpy==1.26.4 sklearn-pandas==2.2.0 torch==2.5.1+cu121

# DistilBERT Hyperparameter Grid Search for Classifying Code Comment Data 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import numpy as np
import datasets
import time
import wandb
import random
import torch

key_file = r'C:\Development\TactitalTensorsFinalProject\WANDB_API_KEY.txt' # Set the variable to a txt file containing your WandB API key

with open(key_file, "r") as f: # Retrieve key from file
    api_key = f.read().strip()

# Log into WandB with the API key
wandb.login(key=api_key)

# Initialize WandB (no need to manually set the WANDB_API_KEY env variable again)


c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torc

True

If using Google Colab, utilize the cell below in replacement of the code following the imports in the cell above.

In [3]:
# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')


## Data

Below, we use the code provided by the NLBSE 2025 competition, to create the classification labels for each language represented in the data set provided data set before using a data loader to retrieve the data set.  

In [4]:
langs = ['java', 'python', 'pharo']

# langs = ['java'] # Using Java as the only language for testing purposes.
# langs = ['python']
# langs = ['pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = datasets.load_dataset('NLBSE/nlbse25-code-comment-classification')

Below we display the loaded dataset to gain an idea of the data we will be working with. Here we can see the the number of samples for each language, along with the sample features. The data set seems to be pre-split into train and test features for each language. In the following cell, we print one sample from the data set for the Java language.   

In [5]:
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [6]:
ds['java_test'][0]

{'index': 5,
 'class': 'AbstractContractGetFileStatusTest.java',
 'comment_sentence': 'accept everything.',
 'partition': 1,
 'combo': 'accept everything. | AbstractContractGetFileStatusTest.java',
 'labels': [0, 0, 1, 0, 0, 0, 0]}

In [7]:
model_name = 'distilbert/distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

The next few functions are to preprocess the trainng and validation sets, and to let the Trainer class to evaluate how well the trainng is going after each epoch.

In [8]:
# To tokenize the text in the 'combo' column of the training dataset of each language.

def tokenize_dataset(examples):
  return tokenizer(examples['combo'], truncation=True, padding="max_length", max_length=128)

In [9]:
# Tokenizes the text 'combo' column of the dataset, changes the values of the
# labels column to float instead of int, and sets to format of the lables column
# to torch tesnsors, which is required by the Trainer class.

def preprocess_dataset(input_dataset):
  processed_dataset = input_dataset.map(tokenize_dataset, batched=True, load_from_cache_file=False)
  processed_dataset = processed_dataset.cast_column("labels", datasets.features.Sequence(datasets.features.Value("float32")))
  processed_dataset.set_format('pt')
  return processed_dataset

## Evaluating Model Predictions

The compute_metrics function calculates precision, recall, and F1 score for multi-label classification tasks. It takes the predictions and true labels, computes the metrics for each class, and appends the results to a global DataFrame, scores. It returns the average precision, recall, and F1 score for all categories.

In [ ]:
scores = pd.DataFrame(columns=['model', 'lan', 'cat', 'precision', 'recall', 'f1'])

def compute_metrics(eval_pred, lang, categories, mod_name):
    global scores
    scores.drop(scores[scores['lan'] == lang].index, inplace=True)

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1)) # True Positive
        fp = np.sum((y_true == 0) & (y_pred == 1)) # False Positive
        fn = np.sum((y_true == 1) & (y_pred == 0)) # False Negative

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'model': mod_name,
            'lan': lang,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame([m for m in metrics_list if m['model'] == mod_name])
    scores = pd.concat([scores, temp_scores], ignore_index=True)
    
    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

The function, measure_runtime_and_flops measures the runtime and FLOPs of the model during inference. It uses PyTorch's profiler to calculate the FLOPs and tracks the time taken for each batch in the dataset, returning the average runtime and FLOPs.

In [11]:
def measure_runtime_and_flops(trainer, dataset):
    flops = 0
    runtime = 0

    for i in range(10):
        for batch in dataset:
            inputs = {key: val.unsqueeze(0).to(trainer.model.device) for key, val in batch.items() if key in ['input_ids', 'attention_mask']}
            with torch.profiler.profile(with_flops=True, activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]) as prof:
                start_time = time.time()
                _ = trainer.model(**inputs)
                end_time = time.time()

            runtime += (end_time - start_time)
            flops += sum(k.flops for k in prof.key_averages()) / 1e9 # Convert Flops to GFLOPs

    avg_runtime = runtime / 10
    avg_flops = flops / 10
    
    return avg_runtime, avg_flops, flops, runtime

In [12]:
def compute_metrics_wrapper(mod_name):
    def inner(eval_pred):
        return compute_metrics(eval_pred, lang, labels[lang], mod_name)
    return inner

The score function calculates a weighted score for model performance based on the average F1 score, runtime, and FLOPs. It applies a weighted sum, prioritizing F1 score, while penalizing models for exceeding the maximum allowed runtime and FLOPs.

In [13]:
max_avg_runtime = 5
max_avg_flops = 5000

def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
            0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
            0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

## Ensuring Reprodicability

Below, we create a function to set the seed in multiple locations to help ensure all calculated scores and models are reproducable for future work. 

In [14]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for CUDA devices
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable to ensure deterministic operations

## Grid Search

Below we test for the best hyperparameters for DistilBERT, by testing different combinations of values for learning rate, number of epochs, batch sizes, and weight of decay. Due to the size of the Java dataset out weighing Python or Pharo's datasets, we use half the number of epochs, rounded down, to save time and ensure each model gets roughly the same amount of training per language. To choose the best model, we seperately evaluate each f1 score, per language per model, and evaluate the f1 score averaged across all three languages per model. The model with the highest f1 score will be evaluated again, against a validation set and a test set of the model, before calculating it's submission score (weighted composite score), using a formula provided by the NBSE2025 competition, and re evaluating it's f1 performance with the test set.

The code performs a grid search to train and evaluate a model on multiple language datasets using different hyperparameter combinations (learning rate, epochs, batch size, weight decay). For each combination, it initializes a model, preprocesses the dataset, splits it into training and validation sets, and trains the model using the Trainer class. Evaluation metrics, particularly F1 scores, are calculated and logged with Weights & Biases. After training, the models and tokenizers are saved, and the average F1 score for each language is stored. The final F1 score across all languages for each model configuration is then recorded in final_model_stats.

In [ ]:
# Training the models for each language and saving the the models and thier
# tokenizers after training.

seed = 27
set_seed(seed)

learning_rates = [5e-5, 5e-6]
epochs = [10, 15, 20]
batch_sizes = [4, 8]
weight_of_decay = [0.01, 0.001]

gs_metrics_dict = {}
final_model_stats = {}


for wd in weight_of_decay:
  for epoch in epochs:
    for bs in batch_sizes:
      for lr in learning_rates:
        print(f'------------------ Starting model ==> learning rate: {lr} epochs: {epoch}, batch size: {bs}, weights of decay: {wd} ---------------------')
        fin_mod_name = f'lr-{lr}_epoch-{epoch}_batchSize-{bs}_weightsOfDecay-{wd}'

        total_flops = 0
        total_time = 0
        total_avg_runtime = 0
        total_avg_flops = 0

        for lang in langs:
          lang_mod_name = f'{lang}_lr-{lr}_epoch-{epoch}_batchSize-{bs}_weightsOfDecay-{wd}'

          wandb.init(
              project="NBSE2025_Distilbert", 
              entity="ayoungren-colostate",
              name=lang_mod_name,
              config={  # Log hyperparameters for each run
                  "learning_rate": lr,
                  "epochs": epoch,
                  "batch_size": bs,
                  "weight_decay": wd,
                  "language": lang
              }
          )
          
          num_labels = len(labels[lang])
          model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

          dataset = preprocess_dataset(ds[f'{lang}_train'])
          train_validation_split = dataset.train_test_split(test_size=0.2, seed=seed)

          train_dataset =  train_validation_split['train']
          validation_dataset = train_validation_split['test']

          training_args = TrainingArguments(
            output_dir=f'./results_{lang_mod_name}',
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_dir=f'./logs_{lang_mod_name}',
            per_device_train_batch_size=bs,
            per_device_eval_batch_size=bs,
              # half as many epochs for Java.
            num_train_epochs=epoch,
            weight_decay=wd,
            learning_rate=lr,
            logging_steps=1000,
            save_total_limit=2,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            seed=seed
          )
          
          trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset= validation_dataset,
            processing_class=tokenizer,
            compute_metrics=compute_metrics_wrapper(fin_mod_name),
          )

          trainer.train()
          metrics = trainer.evaluate()

          print("Evaluation Metrics:", metrics)

          # avg_runtime, avg_flops, total_flops, total_time = measure_runtime_and_flops(trainer, validation_dataset)

            # store each model and succession score by individual language
          lang_mod_scores = scores[(scores['lan'] == lang) & (scores['model'] == fin_mod_name)]
          print('CHECK SCORE', scores[(scores['lan'] == lang) & (scores['model'] == fin_mod_name)])

          lang_avg_f1 = lang_mod_scores['f1'].mean()

          print("Scores: ", scores)

          # sc = round(score(lang_avg_f1, avg_runtime, avg_flops), 2)

          gs_metrics_dict[lang_mod_name] = {
            'lang': lang,
            'avg_f1': lang_avg_f1,
            # 'model': fin_mod_name,
            # 'avg_runtime': avg_runtime,
            # 'avg_flops': avg_flops,
            # 'total_flops': total_flops,
            # 'total_time': total_time,
            # 'succ_score': sc
          }

          print("Metrics Dict:", gs_metrics_dict)
          # total_avg_runtime += avg_runtime
          # total_avg_flops += avg_flops

          trainer.model.save_pretrained(f'./models/{lang_mod_name}')
          tokenizer.save_pretrained(f'./tokenizers/{lang_mod_name}')
          
        
        # store combined average succession score of all languages trained on. (Average Combined score of Java, Python, and Pharo)
        model_stats = scores[scores['model'] == fin_mod_name]
        fin_avg_f1 = model_stats['f1'].mean()
        final_model_stats[fin_mod_name] = fin_avg_f1

------------------ Starting model ==> learning rate: 5e-05 epochs: 10, batch size: 4, weights of decay: 0.01 ---------------------


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 7614/7614 [00:00<00:00, 17531.02 examples/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  7%|▋         | 1003/15230 [00:36<08:06, 29.24it/s]

{'loss': 0.1608, 'grad_norm': 1.0593228340148926, 'learning_rate': 4.6717005909389365e-05, 'epoch': 0.66}


 10%|▉         | 1522/15230 [00:55<08:34, 26.62it/s]C:\Users\ayoun\AppData\Local\Temp\ipykernel_9340\2150236115.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores = pd.concat([scores, temp_scores], ignore_index=True)
                                                    
 10%|█         | 1523/15230 [00:59<08:34, 26.62it/s]

{'eval_loss': 0.1295263171195984, 'eval_f1': 0.6404807062022011, 'eval_precision': 0.6601292916925239, 'eval_recall': 0.6283423946058794, 'eval_runtime': 3.9416, 'eval_samples_per_second': 386.393, 'eval_steps_per_second': 96.662, 'epoch': 1.0}


...

eval/f1,▁▆▇▇▇▇█▇▇▇██▇▇▇█▇▇▇▇█
eval/loss,▂▁▁▂▁▃▂▅▄▅▆▇▆██▇████▂
eval/precision,▁▇▇█▆█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
eval/recall,▁▅▆▆▇▅█▆▇▆██▇▆▇▇█▇▇▇█
eval/runtime,▄▅▄▅▂▃▃▂█▂▄▃▁▅█▅▁▄▃▅▆
eval/samples_per_second,▅▄▄▃▆▅▆▆▁▆▅▅█▃▁▄█▄▅▃▃
eval/steps_per_second,▅▄▄▃▆▅▆▆▁▆▅▅█▃▁▄█▄▅▃▃
train/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▁▁▁▁▁█▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  5%|▌         | 377/7540 [00:15<04:46, 24.97it/s]

{'eval_loss': 0.34614893794059753, 'eval_f1': 0.32863612075874965, 'eval_precision': 0.56889338731444, 'eval_recall': 0.26331916837832153, 'eval_runtime': 0.9664, 'eval_samples_per_second': 390.127, 'eval_steps_per_second': 98.308, 'epoch': 1.0}


RuntimeError: [enforce fail at ..\caffe2\serialize\inline_container.cc:337] . unexpected pos 474380672 vs 474380560

  5%|▌         | 377/7540 [00:27<04:46, 24.97it/s]

## Viewing Model Performance Scores

The first cell shows an error due to an accidental run with a new kernel, but will normally show show the results for the best model for each language, along with their label scores.

In [ ]:
scores 

NameError: name 'scores' is not defined

Below we calculate the average f1, average precision, and average recal scores for the best model in each language.

In [17]:
best_f1_scores = []
best_f1_java_row = scores[scores['lan'] == 'java'].loc[scores[scores['lan'] == 'java']['f1'].idxmax()]
best_f1_scores.append(best_f1_java_row)

best_f1_python_row = scores[scores['lan'] == 'python'].loc[scores[scores['lan'] == 'python']['f1'].idxmax()]
best_f1_scores.append(best_f1_python_row)

best_f1_pharo_row = scores[scores['lan'] == 'pharo'].loc[scores[scores['lan'] == 'pharo']['f1'].idxmax()]
best_f1_scores.append(best_f1_pharo_row)
best_f1_scores

[model        lr-5e-05_epoch-20_batchSize-4_weightsOfDecay-0...
 lan                                                       java
 cat                                                  Ownership
 precision                                             0.964286
 recall                                                0.981818
 f1                                                    0.972973
 Name: 8, dtype: object,
 model        lr-5e-05_epoch-20_batchSize-4_weightsOfDecay-0...
 lan                                                     python
 cat                                                 Parameters
 precision                                             0.910256
 recall                                                0.633929
 f1                                                    0.747368
 Name: 15, dtype: object,
 model        lr-5e-06_epoch-15_batchSize-8_weightsOfDecay-0...
 lan                                                      pharo
 cat                                                 

gs_metric_dicts will show the avg F1 score for every language, per trained model.

In [18]:
gs_metrics_dict

{'java_lr-5e-05_epoch-10_batchSize-4_weightsOfDecay-0.01': {'lang': 'java',
  'avg_f1': 0.8614530647193451},
 'python_lr-5e-05_epoch-10_batchSize-4_weightsOfDecay-0.01': {'lang': 'python',
  'avg_f1': 0.7499024741736606},
 'pharo_lr-5e-05_epoch-10_batchSize-4_weightsOfDecay-0.01': {'lang': 'pharo',
  'avg_f1': 0.6760992595162517},
 'java_lr-5e-06_epoch-10_batchSize-4_weightsOfDecay-0.01': {'lang': 'java',
  'avg_f1': 0.847175679083368},
 'python_lr-5e-06_epoch-10_batchSize-4_weightsOfDecay-0.01': {'lang': 'python',
  'avg_f1': 0.5185931839398215},
 'pharo_lr-5e-06_epoch-10_batchSize-4_weightsOfDecay-0.01': {'lang': 'pharo',
  'avg_f1': 0.3464381423565097},
 'java_lr-5e-05_epoch-10_batchSize-8_weightsOfDecay-0.01': {'lang': 'java',
  'avg_f1': 0.8612716333442287},
 'python_lr-5e-05_epoch-10_batchSize-8_weightsOfDecay-0.01': {'lang': 'python',
  'avg_f1': 0.7202505734514754},
 'pharo_lr-5e-05_epoch-10_batchSize-8_weightsOfDecay-0.01': {'lang': 'pharo',
  'avg_f1': 0.6646768174062629},
 '

final_model_stats shows the average F1 score per trained model architecture.

In [19]:
final_model_stats

{'lr-5e-05_epoch-10_batchSize-4_weightsOfDecay-0.01': 0.763809402132499,
 'lr-5e-06_epoch-10_batchSize-4_weightsOfDecay-0.01': 0.5762243510409606,
 'lr-5e-05_epoch-10_batchSize-8_weightsOfDecay-0.01': 0.7517311590795167,
 'lr-5e-06_epoch-10_batchSize-8_weightsOfDecay-0.01': 0.47797133051203045,
 'lr-5e-05_epoch-15_batchSize-4_weightsOfDecay-0.01': 0.7523858683081371,
 'lr-5e-06_epoch-15_batchSize-4_weightsOfDecay-0.01': 0.6658800533871522,
 'lr-5e-05_epoch-15_batchSize-8_weightsOfDecay-0.01': 0.7681797815506027,
 'lr-5e-06_epoch-15_batchSize-8_weightsOfDecay-0.01': 0.6173060758906344,
 'lr-5e-05_epoch-20_batchSize-4_weightsOfDecay-0.01': 0.7769692605356904,
 'lr-5e-06_epoch-20_batchSize-4_weightsOfDecay-0.01': 0.7154331131665084,
 'lr-5e-05_epoch-20_batchSize-8_weightsOfDecay-0.01': 0.7762857137735598,
 'lr-5e-06_epoch-20_batchSize-8_weightsOfDecay-0.01': 0.6527187327721913,
 'lr-5e-05_epoch-10_batchSize-4_weightsOfDecay-0.001': 0.7563265418690509,
 'lr-5e-06_epoch-10_batchSize-4_weigh

In [25]:
best_f1_score = max(final_model_stats.values())
best_key = [key for key, value in final_model_stats.items() if value == best_f1_score]
(best_key, best_f1_score)

(['lr-5e-05_epoch-20_batchSize-4_weightsOfDecay-0.01'], 0.7769692605356904)

In [26]:
best_f1_score = min(final_model_stats.values())
best_key = [key for key, value in final_model_stats.items() if value == best_f1_score]
(best_key, best_f1_score)

(['lr-5e-06_epoch-10_batchSize-8_weightsOfDecay-0.001'], 0.4777266917629609)